In [23]:
import pandas as pd
import plotly.express as px
import re
from datetime import datetime, date, timedelta

In [2]:
import sqlite3

conn = sqlite3.connect("EXOPLANETS.db")
c = conn.cursor()

In [77]:
def show_line_query_plot(filename, x, y):
    with open(filename, 'r') as sql_file:
        sql = sql_file.read()

    # '.headers on' works for command line execution but causes errors in the python environment
    data = pd.read_sql_query(re.sub('.headers on', '', sql), conn)

    start_date = data["date_added"].iloc[0]
    end_date   = date.fromisoformat(data["date_added"].iloc[-1])
    offset_days = 0
    curr_date = date.fromisoformat(start_date) + timedelta(days=offset_days)
    while curr_date <= end_date:
        if curr_date.isoformat() not in data["date_added"].values:
            data = data.append({"date_added": curr_date.isoformat(), "ROWS_ADDED": 0}, ignore_index=True)
        
        offset_days += 1
        curr_date = date.fromisoformat(start_date) + timedelta(days=offset_days)

    data = data.rename(columns={clmn: clmn.lower() for clmn in data.columns})
    data = data.sort_values(by="date_added")
    return px.line(data, x=x, y=y)

In [78]:
def show_query_plot(filename, x, y):
    with open(filename, 'r') as sql_file:
        sql = sql_file.read()

    # '.headers on' works for command line execution but causes errors in the python environment
    data = pd.read_sql_query(re.sub('.headers on', '', sql), conn)
    data = data.rename(columns={clmn: clmn.lower() for clmn in data.columns})
    return px.bar(data, x=x, y=y)

In [79]:
def show_plot_with_detections(detections_file, data_file, x, y):
    fig = show_query_plot(data_file, x, y)

    with open(detections_file, 'r') as sql_file:
        sql = sql_file.read()
    detections = pd.read_sql_query(re.sub('.headers on', '', sql), conn)
    detections = detections.rename(columns={clmn: clmn.lower() for clmn in detections.columns})
    for _, row in detections.iterrows():
        fig.add_vline(x=row['date_added'], line_color='red')
    
    return fig

In [80]:
show_line_query_plot('queries/freshness/rows-added.sql', 'date_added', 'rows_added').show()

In [6]:
show_query_plot('queries/freshness/days-since-last-update.sql', 'date_added', 'days_since_last_update').show()

In [7]:
show_plot_with_detections(
    'queries/freshness/freshness-detector.sql',
    'queries/freshness/rows-added.sql',
    'date_added',
    'rows_added'
).show()

In [8]:
show_query_plot('queries/distribution/null-rates.sql', 'date_added', 'distance_null_rate').show()

In [9]:
show_query_plot('queries/distribution/null-rates.sql', 'date_added', 'g_null_rate').show()

In [10]:
show_query_plot('queries/distribution/null-rates.sql', 'date_added', 'orbital_period_null_rate').show()

In [11]:
show_query_plot('queries/distribution/null-rates.sql', 'date_added', 'avg_temp_null_rate').show()

In [12]:
show_plot_with_detections(
    'queries/distribution/naive-dist-detector.sql',
    'queries/distribution/null-rates.sql',
    'date_added',
    'avg_temp_null_rate'
).show()

In [15]:
show_plot_with_detections(
    'queries/distribution/dist-query-only-start-dates.sql',
    'queries/distribution/null-rates.sql',
    'date_added',
    'avg_temp_null_rate'
).show()

DatabaseError: Execution failed on sql '

WITH NULL_RATES AS(
  SELECT
    DATE_ADDED,
    CAST(
      SUM(
        CASE
          WHEN AVG_TEMP IS NULL THEN 1
          ELSE 0
        END
      ) AS FLOAT
    ) / COUNT(*) AS AVG_TEMP_NULL_RATE
  FROM
    EXOPLANETS
  GROUP BY
    DATE_ADDED
),

ALL_DATES AS (
  SELECT
    *,
    JULIANDAY(DATE_ADDED) - JULIANDAY(LAG(DATE_ADDED)
      OVER(
        ORDER BY DATE_ADDED
      )
    ) AS DAYS_SINCE_LAST_UPDATE
  FROM
    NULL_RATES
  WHERE
    AVG_TEMP_NULL_RATE > 0.9
)

SELECT
  DATE_ADDED,
  AVG_TEMP_NULL_RATE
FROM
  ALL_DATES
WHERE
  DAYS_SINCE_LAST_ALERT IS NULL OR DAYS_SINCE_LAST_ALERT > 1;
': no such column: DAYS_SINCE_LAST_ALERT